# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 statistics.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%%bash
./refreshdata

---
## Confirmed, deaths, recovered datasets

In [ ]:
import os

import numpy as np
import pandas as pd

from covidvu import utils
from covidvu import vujson

pd.options.mode.chained_assignment = None

# Confirm input file definitions:
( vujson.JH_CSSE_FILE_CONFIRMED, vujson.JH_CSSE_FILE_DEATHS, vujson.JH_CSSE_FILE_RECOVERED, )

In [ ]:
confirmedCases = vujson.allCases()

In [ ]:
newCases = confirmedCases.diff(axis=0)
growthFactor = newCases/newCases.shift(1)

---
## Interactive plots

In [ ]:
from covidvu import visualize
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import fixed

In [ ]:
countries = list(confirmedCases.columns)
multiCountry = widgets.SelectMultiple(
    options=countries,
    value=['!Outside Mainland China'],
    description='Country',
    disabled=False
)
log = widgets.Checkbox(value=False, description='Log scale')

### Growth factor

When the growth factor is consistently below 1, we have moved to the saturation phase.

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(growthFactor), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Growth factor'),
         title=fixed(r'COVID-19 growth factor (new cases today)/(new cases yesterday)')
);

---
&#169; the COVIDvu Contributors.  All rights reserved.